In [15]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np

In [16]:
np.random.seed(1000)

In [17]:
AlexNet = Sequential()

In [18]:
AlexNet.add(Conv2D(filters=96, input_shape=(150,150,3), kernel_size=(11,11), strides=(4,4), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

In [19]:
AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))


In [20]:
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

In [21]:
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

In [22]:
AlexNet.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

In [23]:
#Passing it to a Fully Connected layer
AlexNet.add(Flatten())
# 1st Fully Connected Layer
AlexNet.add(Dense(4096, input_shape=(150,150,3,)))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
# Add Dropout to prevent overfitting
AlexNet.add(Dropout(0.4))

In [24]:
#2nd Fully Connected Layer
AlexNet.add(Dense(4096))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

In [25]:
#3rd Fully Connected Layer
AlexNet.add(Dense(1000))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

In [26]:
#Output Layer
AlexNet.add(Dense(20))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('softmax'))

In [27]:
#Model Summary
AlexNet.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 38, 38, 96)        34944     
_________________________________________________________________
batch_normalization_9 (Batch (None, 38, 38, 96)        384       
_________________________________________________________________
activation_9 (Activation)    (None, 38, 38, 96)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 19, 19, 96)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 19, 19, 256)       614656    
_________________________________________________________________
batch_normalization_10 (Batc (None, 19, 19, 256)       1024      
_________________________________________________________________
activation_10 (Activation)   (None, 19, 19, 256)      

In [28]:
# Compiling the model
AlexNet.compile(loss = keras.losses.categorical_crossentropy, optimizer= 'adam', metrics=['accuracy'])

In [30]:
#Accessing dataset from google drive
from google.colab import drive 
drive.mount('/mntDrive')
dataPath='/mntDrive/My Drive/cas_peal/'
train_dir = dataPath+'train'
val_dir = dataPath+'test'

Drive already mounted at /mntDrive; to attempt to forcibly remount, call drive.mount("/mntDrive", force_remount=True).


In [31]:
from keras.preprocessing.image import ImageDataGenerator
# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
                # This is the target directory
                train_dir,
                # All images will be resized to 150x150
                target_size=(150, 150),
                batch_size=20,
              class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
                val_dir,
                target_size=(150, 150),
                batch_size=20,
                class_mode='categorical')

Found 639 images belonging to 20 classes.
Found 60 images belonging to 20 classes.


In [32]:
for data_batch, labels_batch in validation_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (20, 150, 150, 3)
labels batch shape: (20, 20)


In [38]:
history = AlexNet.fit_generator(
train_generator,
epochs=5,
validation_data=validation_generator)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
32/32 [==============================] - 209s 6s/step - loss: 2.5740 - accuracy: 0.2576 - val_loss: 65.8195 - val_accuracy: 0.0500
Epoch 2/5
32/32 [==============================] - 65s 2s/step - loss: 1.4862 - accuracy: 0.6256 - val_loss: 8.3055 - val_accuracy: 0.1167
Epoch 3/5
32/32 [==============================] - 65s 2s/step - loss: 1.1313 - accuracy: 0.7652 - val_loss: 2.4253 - val_accuracy: 0.3333
Epoch 4/5
32/32 [==============================] - 65s 2s/step - loss: 1.0280 - accuracy: 0.8040 - val_loss: 2.6162 - val_accuracy: 0.2667
Epoch 5/5
32/32 [==============================] - 65s 2s/step - loss: 0.8638 - accuracy: 0.8833 - val_loss: 2.7643 - val_accuracy: 0.1833
